This file is used to run the initial api call on Mountain Project to get a dataset of routes organized by the input regions.

In [66]:
from pymongo import MongoClient
import json, re, requests, pprint, urllib
from bs4 import BeautifulSoup
from mp_api import MP
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
key_path = '../data/mp_api_key.txt'
with open(key_path, 'r') as key_text:
    key = key_text.read()

In [7]:
climb_locations = pd.read_table('../data_coll/climb_locations.txt', sep = ' ', header=0, index_col=0)
print(climb_locations)

                             lat         lon  distance  num_routes
place                                                             
squamish_chief         49.676906 -123.152298         5         500
squamish_smoke_bluffs  49.706507 -123.142327         5         500
squamish_murrin_park   49.644247 -123.208615         5         500
squamish_chek          49.907000 -123.162000        10         500
smith                  44.367954 -121.142405        50         500
index                  47.818521 -121.571301        50         500
exits                  47.427568 -121.633499        50         500
vantage                47.027962 -119.963285        50         500
leavenworth            47.542495 -120.725364        50         500
tieton                 46.684000 -120.958000        50         500
mazama                 48.593000 -120.406000        50         500
erie                   48.453000 -122.627000        50         500


In [99]:
client = MongoClient('localhost', 27017)
db = 'mountain_project'
collection = 'regional_routes'
if collection in client[db].list_collection_names():
    (client[db])[collection].drop()

region_db_creation = MP(key, client, db, collection)

In [ ]:
for region in climb_locations.index:
    lat = climb_locations.loc[region]['lat']
    lon = climb_locations.loc[region]['lon']
    dist = climb_locations.loc[region]['distance']
    num_routes = climb_locations.loc[region]['num_routes']
    region_db_creation.api_get_routes_lat_long(region, lat, lon, dist, num_routes)

In [ ]:
pprint.pprint(region_db_creation.collection.find_one())

In [108]:
route_urls = []

for region in climb_locations.index:
    routes = region_db_creation.collection.find_one({'region': region})['data']
    route_urls += [routes[idx]['url'] for idx in range(len(routes))]

with open('../data/route_urls.txt', 'w') as out:
    [out.write(route_url + '\n') for route_url in route_urls]